In [1]:
import openai
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
load_dotenv()

True

In [3]:
openai_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

In [4]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

In [6]:
DATBASE  = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [7]:
retriever=DATBASE.as_retriever(search_kwargs={"k": 2, "score_threshold": 0.5})

In [8]:
results = await retriever.ainvoke("АРО это")

In [9]:
results

[Document(page_content='АРО\nЭто составляющая первого этапа продаж — Открытие в разговоре с\nКлиентом.\n⠀\nСам этап состоит из 3-х подэтапов:\n1. Приветствие\n2. Актуализация\n3. Понимание\n⠀\nПодэтап "Понимание", в свою очередь, состоит из треугольника 🔺 АРО.\n⠀\n🔻Аффинити — это степень симпатии с Клиентом;\n⠀\n🔻Реальность — выравнивание реальности с Клиентом. Важно, чтобы\nфокус внимания Клиента был на менеджере по продажам и Клиент\nпонимал, что сейчас происходит и будет происходить в диалоге.\n', metadata={'source': 'Исходниики/2 Коммерческое отделение/Отдел продаж/7 Обучающие материалы/По продажам/АРО _ База знаний _ Platrum.pdf', 'page': 0}),
 Document(page_content='треугольника АРО и использовать его.\nПри чем, использовать везде: и в работе, и в общении с друзьями, и в\nсемье.\nЧто такое треугольник АРО?\nЗачем нам это знать?\nЗачем нам вообще что-то знать?\nКогда-то, Ньютону упало яблоко на голову, и после этого он вывел закон и\nформулу тяготения. И, как ни странно, этот зако

In [10]:
results[0].metadata

{'source': 'Исходниики/2 Коммерческое отделение/Отдел продаж/7 Обучающие материалы/По продажам/АРО _ База знаний _ Platrum.pdf',
 'page': 0}

In [11]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [73]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты полезный асистент, дай ответ исходя из контекста: {context}"),
        ("human", "{user_input}"),
    ]
)

In [68]:
documents = RunnableParallel(
    {"context": retriever, "user_input": RunnablePassthrough()}
)

In [69]:
await documents.ainvoke("Что такое АРО")

{'context': [Document(page_content='АРО\nЭто составляющая первого этапа продаж — Открытие в разговоре с\nКлиентом.\n⠀\nСам этап состоит из 3-х подэтапов:\n1. Приветствие\n2. Актуализация\n3. Понимание\n⠀\nПодэтап "Понимание", в свою очередь, состоит из треугольника 🔺 АРО.\n⠀\n🔻Аффинити — это степень симпатии с Клиентом;\n⠀\n🔻Реальность — выравнивание реальности с Клиентом. Важно, чтобы\nфокус внимания Клиента был на менеджере по продажам и Клиент\nпонимал, что сейчас происходит и будет происходить в диалоге.\n', metadata={'source': 'Исходниики/2 Коммерческое отделение/Отдел продаж/7 Обучающие материалы/По продажам/АРО _ База знаний _ Platrum.pdf', 'page': 0}),
  Document(page_content='треугольника АРО и использовать его.\nПри чем, использовать везде: и в работе, и в общении с друзьями, и в\nсемье.\nЧто такое треугольник АРО?\nЗачем нам это знать?\nЗачем нам вообще что-то знать?\nКогда-то, Ньютону упало яблоко на голову, и после этого он вывел закон и\nформулу тяготения. И, как ни стран

In [70]:
chain = documents | prompt_template | model

In [31]:
res = await chain.ainvoke("АРО это")  

In [36]:
res

AIMessage(content='АРО - это треугольник, используемый в первом этапе продаж для лучшего понимания клиента. В данном случае, АРО состоит из следующих компонентов: Аффинити (степень симпатии с клиентом) и Реальность (выравнивание реальности с клиентом).', response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 1137, 'total_tokens': 1235}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-36aa307d-1c18-413c-8d7c-f1d64da228f1-0')